# Face Predictions using DeepFace

In [ ]:
from pathlib import Path
from pprint import pprint

import cv2 as cv
import joblib
import pandas as pd
from tqdm import tqdm

from deepface import DeepFace

import matplotlib
from matplotlib import pyplot as plt

In [ ]:
from saac.utils import cv_imshow, color_show, quadrant_bboxes, crop_bbox, draw_bbox
from saac.models import IdentityClassifier, CalibratedGenderModel
from saac.process import ImageEqualizer, SkinColorExtractor, MidJourneyProcessor

In [ ]:
midjourney_raw_root = Path('../../data/images/raw')

img_path = list(midjourney_raw_root.glob('*.png'))[0]

img_2x2 = cv.imread(str(img_path))
cv_imshow(img_2x2)

In [ ]:
for i, bbox in enumerate(quadrant_bboxes(img_2x2.shape[:2])):
    draw_bbox(img_2x2, bbox, text=f'{i}', color=(0, 255, 255))

cv_imshow(img_2x2)

In [ ]:
quadrant = 2
img_quad = crop_bbox(img_2x2, quadrant_bboxes(img_2x2.shape[:2])[quadrant])
cv_imshow(img_quad)

In [ ]:
processor = MidJourneyProcessor()

predictions = processor.quadrant_predictions(img_2x2)
pprint(predictions[quadrant])

In [ ]:
img_face = crop_bbox(img_2x2, predictions[quadrant]['bbox'])
color = predictions[quadrant]['skin color']

fig, ax = plt.subplots(1, 2, figsize=(6, 3))
cv_imshow(img_face, ax=ax[0])
color_show(color, ax=ax[1])

In [ ]:
df_default_models = {
    'age': DeepFace.build_model('Age'),
    'gender': DeepFace.build_model('Gender'),
    'emotion': DeepFace.build_model('Emotion'),
    'race': DeepFace.build_model('Race')
}

calibrated_model_path = Path('../../models/gender_model_default_calibrated.joblib')
calibrated_clf = joblib.load(calibrated_model_path)

calibrated_gender_model = CalibratedGenderModel(
    gender_model=df_default_models['gender'],
    calibrator=calibrated_clf
)

In [ ]:
def prompt_extract(path: str) -> str:
    return ' '.join(path.split('_')[1:-1])

In [ ]:
# gender_model = df_default_models['gender']
gender_model = calibrated_gender_model

kwargs = {
    'equalizer': True,
    'actions': ('gender', 'skin'),
    'models': { 'gender': gender_model },
    'detector_backend': 'mtcnn'
}

all_predictions = []

for quad_path in tqdm(list(midjourney_raw_root.glob('*.png'))):
    quad_image = cv.imread(str(quad_path))
    prompt = prompt_extract(quad_path.stem)
    quad_predictions = [p if p else {} for p in processor.quadrant_predictions(quad_image, **kwargs)]
    for idx, pred in enumerate(quad_predictions):
        pred['image'] = quad_path.stem
        pred['quadrant'] = idx
        pred['prompt'] = prompt
    all_predictions.extend(quad_predictions)

In [ ]:
results_df = pd.json_normalize(all_predictions)

lead_cols = [
    'prompt',
    'image',
    'quadrant',
    'bbox'
]

results_df = results_df.reindex(columns=lead_cols+[col for col in results_df.columns if col not in lead_cols])
results_df

In [ ]:
results_df.to_csv(Path('./midjourney_deepface_calibrated_equalized.csv'), index=False)